In [1]:
from bs4 import BeautifulSoup
! pip install geocoder
import geocoder
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

    100% |████████████████████████████████| 102kB 11.0MB/s 
distributed 1.21.8 requires msgpack, which is not installed.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         220 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.17.0-py_0      conda-forge

The following packages will be UPDATED:

    certifi:       2018.8.24-py36_1 conda-forge --


####  scrape the Wikipedia page  to get the Toronto neighborhood information 
####  transform the information into a dataframe

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
article=source.text

In [3]:
bs=BeautifulSoup(article,'lxml')
mytable=bs.find ('table',class_='wikitable sortable')
mycontent=[]
i=0
for tdtext in mytable.find_all('td'):
    mycontent.append(tdtext.text)
c1=mycontent[0:len(mycontent):3]
c2=mycontent[1:len(mycontent):3]
c3=mycontent[2:len(mycontent):3]
df=pd.DataFrame({'Postcode':c1,'Borough':c2,'Neighbourhood':c3})
df=df[['Postcode','Borough','Neighbourhood']]
dfcopy=df.copy()#copy dataframe
df['Neighbourhood']
neihood=[]
for str in df['Neighbourhood']:
    neihood.append(str[-len(str):-1])   
df['Neighbourhood']=pd.Series(neihood)
df1=df.copy()#copy dataframe
df2=df[(df['Borough']!='Not assigned')]
df3=df2[(df2['Borough']!='Not assigned')]
df3copy=df.copy()#copy dataframe
df4=df3.groupby('Postcode')
dfnew = pd.DataFrame(columns = ["Postcode", "Borough", "Neighbourhood"]) #创建一个空的dataframe
for name,group in df4:
    
    str1=''
    for str in group['Neighbourhood']:
        str1=str1+str+","
    str2=str1[-len(str1):-1]
    s = pd.Series({'Postcode':name, 'Borough':group.iloc[0][1], 'Neighbourhood':str2})
    dfnew = dfnew.append(s, ignore_index=True)
    
dfnew.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
dfnewcopy=dfnew.copy()
dfnewcopy1=dfnew.copy()
postcode=dfnew['Postcode']
postcode[0]
latitude=[]
longitude=[]
for postcode in dfnewcopy1['Postcode']:  
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates    
    while(lat_lng_coords is None):
          g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
          lat_lng_coords = g.latlng
    latitude1 = lat_lng_coords[0]
    longitude1 = lat_lng_coords[1]
    latitude.append(latitude1)
    longitude.append(longitude1)
dfnewcopy1['Latitude']=pd.Series(latitude)
dfnewcopy1['Longitude']=pd.Series(longitude)
dfnewcopy1.head()

In [8]:
del dfnewcopy1['Postcode']

#### display the dataframe 'neighborhoods2' of the Toronto neighborhood information

In [13]:
neighborhoods2=dfnewcopy1.copy()
neighborhoods2.shape
neighborhoods2.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood]],Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
CLIENT_ID = 'D4D2QBZKG211KJFWL5KNP3V1IX0BIT2RLDTAJ3ZVSM0CZ1CR' # your Foursquare ID
CLIENT_SECRET = 'NP5BZHJYQC3Z3E2554YSGWSTI0AZSUSLHKILTZLDZ1SGBFAR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D4D2QBZKG211KJFWL5KNP3V1IX0BIT2RLDTAJ3ZVSM0CZ1CR
CLIENT_SECRET:NP5BZHJYQC3Z3E2554YSGWSTI0AZSUSLHKILTZLDZ1SGBFAR


#### get the 100 venues for  each neighborhood throgh FourSquare API

In [ ]:
venues_list=[]
radius=500
#Boroughnames=neighborhoods['Borough']
#Countrynames=neighborhoods['Country']
#Citynames=neighborhoods['City']
#names=Queens_data['Neighborhood']
names=neighborhoods2['Neighbourhood']
latitudes=neighborhoods2['Latitude']
longitudes=neighborhoods2['Longitude']
LIMIT=100
for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    


In [48]:
nearby_venues.shape

(2234, 7)

In [18]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


#### compute the tatal number of categories of all veneus for each neighborhood

In [19]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))#

There are 272 uniques categories.


In [20]:
nearby_venues[['Venue Category']].shape

(2234, 1)

#### generate the dummies for all categories of venues

In [21]:
# one hot encoding
manhattan_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.shape

(2234, 272)

#### group according to different neighborhood  and compute the tatal number of each category for each neighborhood.

In [22]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').sum().reset_index()
manhattan_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0,1,0,0,0,0,0,0,0,4,0,0,0,1,1,0,2,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,1

In [23]:
manhattan_grouped.shape

(100, 272)

#### compute the  total number of category whose name contains 'American Restaurant'
#### compute the  total number of category whose name contains 'Restaurant'
#### compute the  total number of category which don't belong to restaurant

In [24]:
list_name1=[]
list_name2=[]
list_name3=[]
for name in manhattan_grouped.columns[1:]:
    if(name.find('American Restaurant')!=-1):
        list_name1.append(name)
    elif(name.find('Restaurant')!=-1):
        list_name2.append(name)
    else:
        list_name3.append(name)
print(len(list_name1))
print(len(list_name2))
print(len(list_name3))

3
50
218


In [25]:
sorted_columns=list_name1[0:]+list_name2[0:]+list_name3[0:]
df=manhattan_grouped[sorted_columns]
#df['']
df['Neighborhood']=manhattan_grouped['Neighborhood']
df.head(2)
#df.shape
fixed_columns = [df.columns[-1]] + list(df.columns[:-1])
df = df[fixed_columns]
df.shape

(100, 272)

#### add three columns 'totals1','totals2','totals3'
#### totals1 is to compute the total of venues whose category is 'American Restaurant' for each neighborhood
#### totals2 is to compute the total of venues whose category is 'Restaurant' for each neighborhood
#### totals3 is to compute the total of venues whose category is 'not restaurant' for each neighborhood

In [26]:
df.head(2)
df.shape
df['totals1']=0
df['totals2']=0
df['totals3']=0
sum2=0
sum1=0
for i in range(len(list_name1)):
    df['totals1']=df['totals1']+df.iloc[:,i+1]
for i in range(len(list_name2)):
    df['totals2']=df['totals2']+df.iloc[:,i+1+len(list_name1)]
for i in range(len(list_name3)):
    df['totals3']=df['totals3']+df.iloc[:,i+1+len(list_name2)]
    

In [27]:
df.shape
df.columns
columns=[]
df3=df.copy()

In [30]:
fixed_columns = [df.columns[0]] + list(df.columns[-3:-1])+[df.columns[-1]]
df2 = df[fixed_columns]
fixed_columns

['Neighborhood', 'totals1', 'totals2', 'totals3']

#### find those neighborhoods in which there are no American restaurant and restaurant
#### in other word, find the dataframe which meets totals1 equals 0 and totals2 equals 0

In [50]:
list3=[]
df_total3=df2[((df2['totals1']==0) & (df2['totals2']==0) )].reset_index(drop=True)
filtered=df_total3.copy()
filtered
df_total3

,Neighborhood,totals1,totals2,totals3
0,Agincourt,0,0,4
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0,0,2
2,"Alderwood,Long Branch",0,0,9
3,"Birch Cliff,Cliffside West",0,0,4
4,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0,0,6
5,"CFB Toronto,Downsview East",0,0,4
6,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0,0,13
7,"Clairlea,Golden Mile,Oakridge",0,0,8
8,"Cloverdale,Islington,Martin Grove,Princess Gar...",0,0,2
9,Davisville North,0,0,7


#### based on the above obtained dataframe, find the neighorhoods in which there are the most venues which are not restaurants,
#### in other words, find dataframe which meets totals3 equals the max.

In [51]:
sorted_filtered=filtered[filtered['totals3']==filtered['totals3'].max()].reset_index(drop=True)
sorted_filtered

,Neighborhood,totals1,totals2,totals3
0,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0,0,13


In [36]:
myneighborhoodstr=sorted_filtered.iloc[0]['Neighborhood']
print(myneighborhoodstr)

CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara


In [37]:
myneighbor=neighborhoods2[neighborhoods2['Neighbourhood']==myneighborhoodstr]

In [39]:
myneighbor.columns

Index(['Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

#### get the latitude and longitude of Toronto city

In [40]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [42]:
import folium # map rendering library

#### create map of Toronto using latitude and longitude values and add the neighborhoods found by us to the map

In [44]:

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(myneighbor['Latitude'], myneighbor['Longitude'], myneighbor['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto